Libraries

In [ ]:
!pip install geopandas

In [1]:
import geopandas as gpd
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from datetime import datetime,date 

from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import KNNImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

print('Libraries imported succesfully')

Libraries imported succesfully


Maps

In [2]:
change_type_map = {'Demolition': 0, 'Road': 1, 'Residential': 2, 'Commercial': 3, 'Industrial': 4,
       'Mega Projects': 5}


change_status_date0_map = {'Construction Done': 10, 'Construction Midway': 1, 
                           'Prior Construction': 2, 'Materials Dumped': 3, 'Land Cleared': 4, 
                           'Operational': 5, 'Construction Started': 6, 'Greenland': 7, 
                           'Excavation': 8, 'Materials Introduced': 9, 0: 0}

print('Maps seted succesfully')

Maps seted succesfully


In [3]:
## Read csvs
train_path = 'train.geojson'
test_path = 'test.geojson'
train_df = gpd.read_file(train_path, index_col=0)
test_df = gpd.read_file(test_path, index_col=0)

print('dataset imported succesfully')

dataset imported succesfully


Dealing with null values

In [4]:
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)

print('all nul values are set to zero')

all nul values are set to zero


Debugging cell


In [5]:
# L = list(train_df.items())
# print(type(train_df))

Training dataset preprocessing

In [6]:
urban_type = ['Sparse Urban', 'Rural', 'Urban Slum', 'Industrial', 'Dense Urban', 'N,A']
geography_type = ['Barren Land', 'Dense Forest', 'River', 'Coastal', 'Desert', 'Grass Land', 'Hills', 'Snow', 'Lakes', 'Sparse Forest', 'Farms', 'N,A']

train_urb = []
train_geo = []

URB = list(train_df['urban_type'])
GEO = list(train_df['geography_type'])

for feat in URB :
    k = []
    for fu in urban_type :
        k += [1 if fu in feat  else 0]
    train_urb.append(k)

for feat in GEO :
    k = []
    for fu in geography_type :
        k += [1 if fu in feat  else 0]
    train_geo.append(k)




In [7]:
def calcul_time (date_string) :
    if date_string==0:
        return 0
    else:
        # convert the date string to a date object
        date_object = datetime.strptime(date_string, "%d-%m-%Y").date()
        # convert the date object to the number of days
        number_of_days = (date_object - date(2000,1,1)).days
        return(number_of_days)

def change_columns(table):
    n,p=table.shape
    train_x_time=np.empty((n,1))
    Columns=table.columns
    for i in range(1,p-1):
        l=table[Columns[i]]
        if Columns[i] in ['date1','date2','date3','date4','date5']:
            train_xi=np.asarray(l.apply(lambda x: calcul_time(x))).reshape(-1,1)
            train_x_time=np.concatenate((train_x_time,train_xi),axis=1)
    return train_x_time

In [ ]:
train_geo1 = np.asarray(train_df[['geometry']].area).reshape(-1,1)
# train_geo2 = np.asarray(train_df[['geometry']].bounds).reshape(-1,1)
train_geo3 = np.asarray(train_df[['geometry']].length).reshape(-1,1)
train_geo4 = np.asarray(train_df[['geometry']].centroid.x).reshape(-1,1)
train_geo5 = np.asarray(train_df[['geometry']].centroid.y).reshape(-1,1)
# train_geo5 = np.asarray(train_df[['geometry']].interiors).reshape(-1,1)
# train_geo6 = np.asarray(train_df[['geometry']].exterior).reshape(-1,1)
# train_geo7 = np.asarray(train_df[['geometry']].representative_point).reshape(-1,1)
# train_geo8 = np.asarray(train_df[['geometry']].envelope).reshape(-1,1)
# train_geo9 = np.asarray(train_df[['geometry']].convex_hull).reshape(-1,1)
train_geo0 = np.asarray(train_df[['geometry']].boundary.length).reshape(-1,1)

train_x9 = np.concatenate((train_geo1,train_geo3,train_geo4,train_geo5,train_geo0),axis=1)

In [35]:
train_x0 = np.asarray(train_df['change_status_date0'].apply(lambda x: change_status_date0_map[x])).reshape(-1,1)
train_x1 = np.asarray(train_df['change_status_date1'].apply(lambda x: change_status_date0_map[x])).reshape(-1,1)
train_x2 = np.asarray(train_df['change_status_date2'].apply(lambda x: change_status_date0_map[x])).reshape(-1,1)
train_x3 = np.asarray(train_df['change_status_date3'].apply(lambda x: change_status_date0_map[x])).reshape(-1,1)
train_x4 = np.asarray(train_df['change_status_date4'].apply(lambda x: change_status_date0_map[x])).reshape(-1,1)

# train_x5 = np.asarray(train_df['urban_type'].apply(lambda x: urban_type_map[x])).reshape(-1,1)


train_x_time = change_columns(train_df)

train_x=np.concatenate((train_urb,train_geo,train_x9,train_x0,train_x1,train_x2,train_x3,train_x4,train_x_time),axis=1)

print('maps applied succesfully to trainning dataset')

maps applied succesfully to trainning dataset


In [36]:
FEATURES = ['img_red_mean_date1', 'img_green_mean_date1', 'img_blue_mean_date1', 'img_red_std_date1', 'img_green_std_date1', 'img_blue_std_date1', 'img_red_mean_date2', 'img_green_mean_date2', 'img_blue_mean_date2', 'img_red_std_date2', 'img_green_std_date2', 'img_blue_std_date2', 'img_red_mean_date3', 'img_green_mean_date3', 'img_blue_mean_date3', 'img_red_std_date3', 'img_green_std_date3', 'img_blue_std_date3', 'img_red_mean_date4', 'img_green_mean_date4', 'img_blue_mean_date4', 'img_red_std_date4', 'img_green_std_date4', 'img_blue_std_date4', 'img_red_mean_date5', 'img_green_mean_date5', 'img_blue_mean_date5', 'img_red_std_date5', 'img_green_std_date5', 'img_blue_std_date5']

In [37]:
for f in FEATURES :
    newtrain = np.asarray(train_df[[f]]).reshape(-1,1)
    train_x=np.concatenate((train_x,newtrain),axis=1)


In [38]:
## Filtering column "mail_type"
train_y = train_df['change_type'].apply(lambda x: change_type_map[x])

print('output selected succesfully')

output selected succesfully


Test dataset

In [39]:
test_urb = []
test_geo = []

URB = list(test_df['urban_type'])
GEO = list(test_df['geography_type'])

for feat in URB :
    k = []
    for fu in urban_type :
        k += [1 if fu in feat  else 0]
    test_urb.append(k)

for feat in GEO :
    k = []
    for fu in geography_type :
        k += [1 if fu in feat  else 0]
    test_geo.append(k)

In [ ]:
test_geo1 = np.asarray(test_df[['geometry']].area).reshape(-1,1)
# test_geo2 = np.asarray(test_df[['geometry']].bounds).reshape(-1,1)
test_geo3 = np.asarray(test_df[['geometry']].length).reshape(-1,1)
test_geo4 = np.asarray(test_df[['geometry']].centroid.x).reshape(-1,1)
test_geo5 = np.asarray(test_df[['geometry']].centroid.y).reshape(-1,1)
# test_geo6 = np.asarray(test_df[['geometry']].exterior).reshape(-1,1)
# test_geo7 = np.asarray(test_df[['geometry']].representative_point).reshape(-1,1)
# test_geo8 = np.asarray(test_df[['geometry']].envelope).reshape(-1,1)
# test_geo9 = np.asarray(test_df[['geometry']].convex_hull).reshape(-1,1)
test_geo0 = np.asarray(test_df[['geometry']].boundary.length).reshape(-1,1)

test_x9 = np.concatenate((test_geo1, test_geo3, test_geo4, test_geo5, test_geo0), axis=1)

In [54]:
test_geo6 = np.asarray(test_df[['geometry']].exterior).reshape(-1,1)
print(test_geo6[0])

[<LINEARRING (2.903 35.431, 2.903 35.431, 2.903 35.431, 2.903 35.431, 2.903 3...>]


In [42]:
test_x0 = np.asarray(test_df['change_status_date0'].apply(lambda x: change_status_date0_map[x])).reshape(-1,1)
test_x1 = np.asarray(test_df['change_status_date1'].apply(lambda x: change_status_date0_map[x])).reshape(-1,1)
test_x2 = np.asarray(test_df['change_status_date2'].apply(lambda x: change_status_date0_map[x])).reshape(-1,1)
test_x3 = np.asarray(test_df['change_status_date3'].apply(lambda x: change_status_date0_map[x])).reshape(-1,1)
test_x4 = np.asarray(test_df['change_status_date4'].apply(lambda x: change_status_date0_map[x])).reshape(-1,1)

# test_x5 = np.asarray(test_df['urban_type'].apply(lambda x: urban_type_map[x])).reshape(-1,1)



test_x_time = change_columns(test_df)

test_x = np.concatenate((test_urb, test_geo, test_x9, test_x0, test_x1, test_x2, test_x3, test_x4, test_x_time), axis=1)


print('maps applied succesfully to testing dataset')

maps applied succesfully to testing dataset


In [43]:
for f in FEATURES :
    newtest = np.asarray(test_df[[f]]).reshape(-1,1)
    test_x = np.concatenate((test_x,newtest), axis=1)

In [44]:
print (train_x.shape, train_y.shape, test_x.shape)

(296146, 63) (296146,) (120526, 63)


Model

Random Forest

In [46]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Create an instance of the RandomForestClassifier
clf = RandomForestClassifier(n_estimators=200, random_state=42)

# Fit the model on the training data
clf.fit(train_x, train_y)

# Make predictions on the testing data
pred_y = clf.predict(test_x)
